In [1]:
from pathlib import Path
import sys  
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [23]:
from libs.Recommenders.MatrixFactorization.SVDFeatureRecommender import SVDFeature
from libs.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
import numpy as np
from operator import itemgetter

In [9]:
%load_ext autoreload
%autoreload 2


In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 174 ( 0.5%) Users that have less than 1 test interactions


In [6]:
def save_history():
    pass

In [7]:
svd_model = SVDFeature(URM_train=URM_train, ICM = ICM_all)

In [22]:
svd_model._compute_item_score(np.array([1000]))

FileNotFoundError: [Errno 2] No such file or directory: 'svd_feature_infer'

In [18]:
svd_model.fit(temp_file_folder="'./result_experiments/__Temp_SVDFeature/'")

SVDFeature: Using Temp folder ''./result_experiments/__Temp_SVDFeature/''
SVDFeature: Writing input file in feature format
SVDFeature: Fit starting


FileNotFoundError: [Errno 2] No such file or directory: 'svd_feature_train'

In [17]:
!which svd_feature_train

svd_feature_train not found


In [15]:
!ls

'.                                   svd_feature_train
__init__.py                          tw-01-data_preprocessing.ipynb
result_experiments                   tw-02-top_popular.ipynb
sf-01-item-knn-cbf-cbfr.ipynb        tw-03-content_based_filtering_.ipynb
sf-03-MF_methods.ipynb               tw-04-collaborative_filtering.ipynb
sf-predict.ipynb                     tw-05-hybrid_cbf_cf.ipynb
sf-slim.ipynb


In [7]:
MAP = []

In [ ]:

URM_train_coo = URM_train.tocoo()


for topK_ in [50, 100, 150, 200]:
    for l1_ratio_ in np.linspace(0.001, 1, 10):
        for alpha_ in np.linspace(0.00001, 0.0005, 3):
            slim_model = SLIMElasticNetRecommender(URM_train_coo, verbose=True)
            slim_model.fit(l1_ratio=l1_ratio_, alpha = alpha_, topK = topK_)

            result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)
            MAP.append(('topK={}'.format(topK_), 'l1_ratio={}'.format(l1_ratio_), 'alpha={}'.format(alpha_), result_dict["MAP"][10]))

            print(MAP[-1])
            

In [24]:
pure_svd = PureSVDRecommender(URM_train, verbose=True)

In [25]:
pure_svd.fit()

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 1.03 sec


In [27]:
result_dict, _ = evaluator_validation.evaluateRecommender(pure_svd)

EvaluatorHoldout: Processed 35562 (100.0%) in 20.51 sec. Users per second: 1734


In [28]:
result_dict["MAP"]

cutoff
10    0.020634
Name: MAP, dtype: object